In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import glob
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pathlib
from typing import Type
import json

import jax
import jax.numpy as jnp
jax.config.update("jax_platform_name", "cpu")
#jax.config.update("jax_enable_x64", True)
import equinox as eqx

from mc2.runners.model_setup_jax import get_normalizer
from mc2.data_management import EXPERIMENT_LOGS_ROOT, MODEL_DUMP_ROOT, NORMALIZATION_ROOT, DATA_ROOT, MaterialSet
from mc2.models.jiles_atherton import JAStatic, JAWithGRU
from mc2.models.RNN import GRU
from mc2.model_interfaces.model_interface import ModelInterface, load_model, count_model_parameters
from mc2.utils.model_evaluation import (
    reconstruct_model_from_exp_id, get_exp_ids, store_model_to_file, reconstruct_model_from_file
)

In [ ]:
from mc2.data_management import AVAILABLE_MATERIALS, FINAL_MATERIALS

In [ ]:
exp_ids = sorted(get_exp_ids())
exp_ids

In [ ]:
for exp_id in exp_ids:
    print(exp_id)
    material_name = exp_id.split("_")[0]
    model_type = exp_id.split("_")[1]

    # load legacy model:
    experiment_path = EXPERIMENT_LOGS_ROOT / "jax_experiments"
    with open(experiment_path / f"{exp_id}.json", "r") as f:
        params = json.load(f)["params"]
    
    wrapped_model, data_tuple = reconstruct_model_from_exp_id(exp_id, data_tuple=None)


    # store in new format:
    params["material_name"] = exp_id.split("_")[0]
    params["model_type"] = exp_id.split("_")[1]
    
    store_model_to_file(
        filename=DATA_ROOT / "single_file_models" / f"{exp_id}.eqx",
        wrapped_model=wrapped_model,
        params=params,
    )
    print("-------------------------------------------------------")